# 🐊 UF Athletics Data Collection & Cleaning Notebook

This notebook automatically collects, cleans, and merges data on University of Florida athletics teams (SEC sports only),
focusing on championship seasons and inter-team performance trends.

### Sources used
1. [FloridaGators.com - Official Overview of Championships](https://floridagators.com/sports/2015/12/10/_overview_)
2. [SEC Sports - University of Florida](https://www.secsports.com/school/university-of-florida)
3. [Sports-Reference College Football](https://www.sports-reference.com/cfb/schools/florida/)
4. [Sports-Reference College Basketball](https://www.sports-reference.com/cbb/schools/florida/)
5. [Wikipedia - Florida Gators](https://en.wikipedia.org/wiki/Florida_Gators) (cross-check only)

**Note:** Lacrosse and any non-SEC sports are excluded.


In [22]:
# Install required packages if not already installed
import subprocess
import sys

def install_if_missing(import_name, package_name=None):
    """Install package if import fails. package_name defaults to import_name."""
    if package_name is None:
        package_name = import_name
    try:
        __import__(import_name)
    except ImportError:
        print(f'Installing {package_name}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])

# Install required packages
install_if_missing('requests')
install_if_missing('pandas')
install_if_missing('bs4', 'beautifulsoup4')
install_if_missing('lxml')
install_if_missing('html5lib')
print('✅ All required packages are installed.')


✅ All required packages are installed.


In [ ]:
import os, re, time
import requests  # type: ignore
import pandas as pd  # type: ignore
from bs4 import BeautifulSoup  # type: ignore
from io import StringIO

OUTPUT_DIR = 'cleaned_data'
os.makedirs(OUTPUT_DIR, exist_ok=True)

HEADERS = {'User-Agent': 'Mozilla/5.0 (compatible; UFDataCollector/1.0)'}

sources = {
    'floridagators_overview': 'https://floridagators.com/sports/2015/12/10/_overview_',
    'secsports_school': 'https://www.secsports.com/school/university-of-florida',
    'sportsref_cfb': 'https://www.sports-reference.com/cfb/schools/florida/',
    'sportsref_cbb': 'https://www.sports-reference.com/cbb/schools/florida/',
    'wikipedia_gators': 'https://en.wikipedia.org/wiki/Florida_Gators',
    'wikipedia_baseball': 'https://en.wikipedia.org/wiki/List_of_Florida_Gators_baseball_seasons'
}
sources

{'floridagators_overview': 'https://floridagators.com/sports/2015/12/10/_overview_',
 'secsports_school': 'https://www.secsports.com/school/university-of-florida',
 'sportsref_cfb': 'https://www.sports-reference.com/cfb/schools/florida/',
 'sportsref_cbb': 'https://www.sports-reference.com/cbb/schools/florida/',
 'sportsref_baseball': 'https://www.sports-reference.com/cbb/schools/florida/',
 'wikipedia_gators': 'https://en.wikipedia.org/wiki/Florida_Gators'}

In [24]:
def read_html_tables(url, match=None, header=0):
    try:
        print(f'Trying pandas.read_html on {url}')
        kwargs = {'header': header}
        if match is not None:
            kwargs['match'] = match
        dfs = pd.read_html(url, **kwargs)
        return dfs
    except Exception as e:
        print('Primary read_html failed:', e)
        try:
            resp = requests.get(url, headers=HEADERS, timeout=20)
            html = resp.text
            kwargs = {'header': header}
            if match is not None:
                kwargs['match'] = match
            dfs = pd.read_html(StringIO(html), **kwargs)
            return dfs
        except Exception as e2:
            print('Fallback failed:', e2)
            return []

In [ ]:
# Example: Scrape Sports-Reference pages for Football and Basketball
cfb_dfs = read_html_tables(sources['sportsref_cfb'])
cbb_dfs = read_html_tables(sources['sportsref_cbb'])

def extract_main_table(dfs):
    if not dfs: 
        print('Warning: No tables found')
        return pd.DataFrame()
    df = max(dfs, key=lambda x: x.shape[0])
    if df.empty:
        print('Warning: Largest table is empty')
        return pd.DataFrame()
    
    # Check if first row contains header-like values (Year, Season, W, L, etc.)
    first_row = df.iloc[0].astype(str).str.lower()
    if any(term in ' '.join(first_row.values) for term in ['year', 'season', 'rk']):
        # First row is actually the header, use it and drop it
        df.columns = df.iloc[0].astype(str)
        df = df.iloc[1:].reset_index(drop=True)
    
    # Clean column names
    df.columns = [str(c).strip().lower() for c in df.columns]
    
    # Remove rows that are all NaN or look like headers
    df = df[~df.iloc[:, 0].astype(str).str.contains('rk|year|season', case=False, na=False)]
    df = df.dropna(how='all').reset_index(drop=True)
    
    return df

cfb_raw = extract_main_table(cfb_dfs)
cbb_raw = extract_main_table(cbb_dfs)

if not cfb_raw.empty:
    cfb_raw.to_csv(f'{OUTPUT_DIR}/sportsref_cfb_raw.csv', index=False)
if not cbb_raw.empty:
    cbb_raw.to_csv(f'{OUTPUT_DIR}/sportsref_cbb_raw.csv', index=False)
print('Saved raw football and basketball tables.')

Trying pandas.read_html on https://www.sports-reference.com/cfb/schools/florida/
Trying pandas.read_html on https://www.sports-reference.com/cbb/schools/florida/
Saved raw football and basketball tables.


In [ ]:
def normalize_season_to_yryr(s):
    """Convert season to 2-digit yr-yr format (e.g., 2024 -> 24-25)"""
    s = str(s).strip()
    
    # If already in 2-digit yr-yr format (25-26), return as is
    m_2digit = re.match(r'(\d{2})-(\d{2})$', s)
    if m_2digit:
        return s
    
    # If in 4-digit yr-yr format (2024-25 or 2024-2025), convert to 2-digit
    m_4digit = re.match(r'(\d{4})-(\d{2,4})', s)
    if m_4digit:
        year1 = int(m_4digit.group(1))
        year2_str = m_4digit.group(2)
        if len(year2_str) == 2:
            # Already 2-digit second year, just convert first
            return f'{year1 % 100:02d}-{year2_str}'
        elif len(year2_str) == 4:
            # Both 4-digit, convert both
            year2 = int(year2_str)
            return f'{year1 % 100:02d}-{year2 % 100:02d}'
    
    # If just a 4-digit year, convert to 2-digit yr-yr format
    m_year = re.match(r'(\d{4})$', s)
    if m_year:
        year = int(m_year.group(1))
        next_year = (year + 1) % 100
        return f'{year % 100:02d}-{next_year:02d}'
    
    # If just a 2-digit year, assume it's the first year
    m_2year = re.match(r'(\d{2})$', s)
    if m_2year:
        year = int(m_2year.group(1))
        next_year = (year + 1) % 100
        return f'{year:02d}-{next_year:02d}'
    
    return s

def clean_sportsref(df, sport):
    """Clean data and return: year, wins, losses, win%, national_championship"""
    if df.empty:
        return pd.DataFrame(columns=['year', 'wins', 'losses', 'win%', 'national_championship'])
    df = df.copy()
    
    # Initialize wins and losses columns
    df['wins'] = pd.NA
    df['losses'] = pd.NA
    
    # Try to find and populate wins column (case-insensitive search)
    for c in df.columns:
        if str(c).lower().strip() in ['w', 'wins']:
            df['wins'] = pd.to_numeric(df[c], errors='coerce')
            break
    
    # Try to find and populate losses column (case-insensitive search)
    for c in df.columns:
        if str(c).lower().strip() in ['l', 'losses']:
            df['losses'] = pd.to_numeric(df[c], errors='coerce')
            break
    
    # Handle season column (case-insensitive search)
    season_col = None
    for c in df.columns:
        if str(c).lower().strip() in ['season', 'year']:
            season_col = c
            break
    
    if season_col:
        df['year'] = df[season_col].astype(str)
    else:
        # Try to use index if it looks like a year
        df['year'] = df.index.astype(str)
    
    # Normalize year to 2-digit yr-yr format (e.g., 24-25)
    df['year'] = df['year'].apply(normalize_season_to_yryr)
    
    # Calculate win percentage
    df['win%'] = df.apply(
        lambda row: (row['wins'] / (row['wins'] + row['losses']) * 100) 
        if pd.notna(row['wins']) and pd.notna(row['losses']) and (row['wins'] + row['losses']) > 0 
        else pd.NA, 
        axis=1
    )
    
    # Round win% to 3 decimal places
    df['win%'] = df['win%'].round(3)
    
    # Check for championship indicators in all columns
    notes_cols = [c for c in df.columns if any(term in str(c).lower() for term in ['note', 'tourney', 'champ', 'ncaa', 'result', 'postseason'])]
    df['national_championship'] = 'no'
    for c in notes_cols:
        if c in df.columns:
            # Check if this row indicates a national championship
            champ_mask = df[c].astype(str).str.contains('champ|national.*champion|won.*ncaa|ncaa.*champion', case=False, na=False, regex=True)
            df.loc[champ_mask, 'national_championship'] = 'yes'
    
    # Filter out rows with no valid wins/losses data
    df = df[df['wins'].notna() & df['losses'].notna()].copy()
    
    # Return columns in order: year, wins, losses, win%, national_championship
    return df[['year', 'wins', 'losses', 'win%', 'national_championship']]

def clean_baseball_wikipedia(df):
    """Clean baseball data from Wikipedia and return: year, wins, losses, ties, win%, national_championship"""
    if df.empty:
        return pd.DataFrame(columns=['year', 'wins', 'losses', 'ties', 'win%', 'national_championship'])
    df = df.copy()
    
    # Flatten multi-level column names if they exist
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(col).strip() for col in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    
    # Initialize columns
    df['wins'] = pd.NA
    df['losses'] = pd.NA
    df['ties'] = 0  # Default to 0 if not found
    
    # Find wins column - look for "Wins" in regular season
    wins_col = None
    for c in df.columns:
        col_lower = str(c).lower().strip()
        if 'win' in col_lower and 'loss' not in col_lower and 'pct' not in col_lower:
            wins_col = c
            break
    
    if wins_col is not None:
        df['wins'] = pd.to_numeric(df[wins_col], errors='coerce')
    
    # Find losses column
    losses_col = None
    for c in df.columns:
        col_lower = str(c).lower().strip()
        if 'loss' in col_lower:
            losses_col = c
            break
    
    if losses_col is not None:
        df['losses'] = pd.to_numeric(df[losses_col], errors='coerce')
    
    # Find ties column
    ties_col = None
    for c in df.columns:
        col_lower = str(c).lower().strip()
        if col_lower in ['t', 'ties', 'tie']:
            ties_col = c
            break
    
    if ties_col is not None:
        df['ties'] = pd.to_numeric(df[ties_col], errors='coerce').fillna(0)
    else:
        df['ties'] = 0
    
    # Find year/season column
    season_col = None
    for c in df.columns:
        col_lower = str(c).lower().strip()
        if col_lower in ['season', 'year']:
            season_col = c
            break
    
    if season_col:
        df['year'] = df[season_col].astype(str)
    else:
        df['year'] = df.index.astype(str)
    
    # Normalize year to 2-digit yr-yr format
    df['year'] = df['year'].apply(normalize_season_to_yryr)
    
    # Calculate win percentage (wins / (wins + losses + ties) * 100)
    df['win%'] = df.apply(
        lambda row: (float(row['wins']) / (float(row['wins']) + float(row['losses']) + float(row['ties'])) * 100) 
        if pd.notna(row['wins']) and pd.notna(row['losses']) and (float(row['wins']) + float(row['losses']) + float(row['ties'])) > 0 
        else pd.NA, 
        axis=1
    )
    
    # Convert to numeric and round
    df['win%'] = pd.to_numeric(df['win%'], errors='coerce')
    df['win%'] = df['win%'].round(3)
    
    # Check for national championship - look in all columns
    df['national_championship'] = 'no'
    for c in df.columns:
        if c in df.columns:
            # Check for championship indicators
            champ_mask = df[c].astype(str).str.contains('champion|CWS|College World Series|won.*ncaa|bold', case=False, na=False, regex=True)
            df.loc[champ_mask, 'national_championship'] = 'yes'
    
    # Filter out rows with no valid wins/losses data
    df = df[df['wins'].notna() & df['losses'].notna()].copy()
    
    if df.empty:
        return pd.DataFrame(columns=['year', 'wins', 'losses', 'ties', 'win%', 'national_championship'])
    
    # Return columns in order: year, wins, losses, ties, win%, national_championship
    return df[['year', 'wins', 'losses', 'ties', 'win%', 'national_championship']]

In [ ]:
# Clean and save football data
cleaned_cfb = clean_sportsref(cfb_raw, 'football')
if not cleaned_cfb.empty:
    cleaned_cfb.to_csv(f'{OUTPUT_DIR}/football.csv', index=False)
    print(f'✓ Football: {len(cleaned_cfb)} seasons saved')
    print('\nFirst 10 rows:')
    print(cleaned_cfb.head(10))
    print(f'\nNational Championships: {(cleaned_cfb["national_championship"] == "yes").sum()}')
else:
    print('⚠ No football data extracted')

print('\n' + '='*60 + '\n')

# Clean and save basketball data
cleaned_cbb = clean_sportsref(cbb_raw, 'basketball')
if not cleaned_cbb.empty:
    cleaned_cbb.to_csv(f'{OUTPUT_DIR}/basketball.csv', index=False)
    print(f'✓ Basketball: {len(cleaned_cbb)} seasons saved')
    print('\nFirst 10 rows:')
    print(cleaned_cbb.head(10))
    print(f'\nNational Championships: {(cleaned_cbb["national_championship"] == "yes").sum()}')
else:
    print('⚠ No basketball data extracted')

print('\n' + '='*60 + '\n')

# Scrape and clean baseball data from Wikipedia
print('Scraping Baseball from Wikipedia...')
print('='*60)
baseball_dfs = read_html_tables(sources['wikipedia_baseball'])

if baseball_dfs:
    # Find the main table (usually the largest one)
    baseball_raw = extract_main_table(baseball_dfs)
    if not baseball_raw.empty:
        baseball_raw.to_csv(f'{OUTPUT_DIR}/wikipedia_baseball_raw.csv', index=False)
        print(f'✓ Baseball raw: {len(baseball_raw)} rows saved')
        
        # Clean baseball data
        cleaned_baseball = clean_baseball_wikipedia(baseball_raw)
        if not cleaned_baseball.empty:
            cleaned_baseball.to_csv(f'{OUTPUT_DIR}/baseball.csv', index=False)
            print(f'\n✓ Baseball: {len(cleaned_baseball)} seasons saved')
            print('\nFirst 10 rows:')
            print(cleaned_baseball.head(10))
            print(f'\nNational Championships: {(cleaned_baseball["national_championship"] == "yes").sum()}')
        else:
            print('⚠ No baseball data extracted after cleaning')
    else:
        print('⚠ No baseball table found')
else:
    print('⚠ Could not scrape baseball data from Wikipedia')

Master dataframe shape: (230, 6)
      sport season  wins  losses  national_champion season_norm
0  football      0   NaN     NaN              False           0
1  football      1   NaN     NaN              False           1
2  football      2   NaN     NaN              False           2
3  football      3   NaN     NaN              False           3
4  football      4   NaN     NaN              False           4


In [ ]:
# Quality check report
print('\n' + '='*60)
print('DATA QUALITY REPORT')
print('='*60)

if not cleaned_cfb.empty:
    print(f'\n📊 Football:')
    print(f'  Total seasons: {len(cleaned_cfb)}')
    print(f'  Date range: {cleaned_cfb["year"].min()} → {cleaned_cfb["year"].max()}')
    print(f'  Valid win% records: {cleaned_cfb["win%"].notna().sum()}')
    print(f'  National Championships: {(cleaned_cfb["national_championship"] == "yes").sum()}')

if not cleaned_cbb.empty:
    print(f'\n📊 Basketball:')
    print(f'  Total seasons: {len(cleaned_cbb)}')
    print(f'  Date range: {cleaned_cbb["year"].min()} → {cleaned_cbb["year"].max()}')
    print(f'  Valid win% records: {cleaned_cbb["win%"].notna().sum()}')
    print(f'  National Championships: {(cleaned_cbb["national_championship"] == "yes").sum()}')

Total rows: 230
Sports: ['football' 'basketball']
Championship seasons flagged: 0
Seasons span: 0 → 99


✅ **Done!**

- Data saved to `/cleaned_data/` folder.
- **football.csv**: Year (yr-yr format), Wins, Losses, Win%, National Championship (yes/no)
- **basketball.csv**: Year (yr-yr format), Wins, Losses, Win%, National Championship (yes/no)
- All data comes from verified official and authoritative public sources.

**CSV Format:**
- Column 1: `year` (e.g., 24-25, 25-26)
- Column 2: `wins`
- Column 3: `losses`
- Column 4: `win%`
- Column 5: `national_championship` (yes/no)

**Next:** Add scraping code for additional sports (baseball, softball, women's basketball, tennis, women's tennis, soccer, gymnastics, volleyball).